In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import traceback


In [2]:
# Load dataset from Google Drive or local files
# from google.colab import drive
# drive.mount('/content/drive')  # Mount Google Drive

# Replace with the path to your dataset
dataset_path = '/content/water_availability_data.csv'
data = pd.read_csv(dataset_path)

# Display basic information
print("Dataset Shape:", data.shape)
print("Dataset Preview:\n", data.head())


Dataset Shape: (1980, 9)
Dataset Preview:
         res_state res_district              res_name  res_year res_month  \
0  Andhra Pradesh       Guntur  Dr.KLRS Pulichintala      2009   january   
1  Andhra Pradesh       Guntur  Dr.KLRS Pulichintala      2009  february   
2  Andhra Pradesh       Guntur  Dr.KLRS Pulichintala      2009     march   
3  Andhra Pradesh       Guntur  Dr.KLRS Pulichintala      2009     april   
4  Andhra Pradesh       Guntur  Dr.KLRS Pulichintala      2009       may   

   res_level  cur_livsto rain_month  rainfall  
0  54.895806    0.251452    january       0.0  
1  54.095000    0.198107   february       0.0  
2  52.902258    0.136903      march       0.3  
3  50.533333    0.064367      april       0.0  
4  47.592581    0.018258        may       0.0  


In [3]:
# Define categorical and numeric columns
categorical_cols = ['res_district', 'res_name', 'res_month', 'rain_month']
numeric_cols = ['res_year', 'res_level', 'cur_livsto', 'rainfall']

# Drop missing values
data = data.dropna()

# Encode categorical variables
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Scale numeric features
scaler_X = MinMaxScaler()
X = data[numeric_cols]
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y = data["cur_livsto"].values.reshape(-1, 1)
y_scaled = scaler_y.fit_transform(y)

# Reshape data for LSTM
def create_lstm_data(X, y, time_steps=1):
    X_lstm, y_lstm = [], []
    for i in range(len(X) - time_steps):
        X_lstm.append(X[i:i+time_steps])
        y_lstm.append(y[i+time_steps])
    return np.array(X_lstm), np.array(y_lstm)

time_steps = 3
X_lstm, y_lstm = create_lstm_data(X_scaled, y_scaled, time_steps)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)


<ipython-input-3-b3e672780096>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
<ipython-input-3-b3e672780096>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
<ipython-input-3-b3e672780096>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [4]:
# Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=32, verbose=1)


Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0285
Epoch 2/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0148
Epoch 3/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0101
Epoch 4/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0093
Epoch 5/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0077
Epoch 6/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0085
Epoch 7/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0093
Epoch 8/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0082
Epoch 9/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0100
Epoch 10/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0081
Epoch 11/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0075
Epoch 12/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0061
Epoch 13/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0061
Epoch 14/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0059
Epoch 15/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0071
Epoch 16/25
36/36 ━━━━━━━━━━━━

In [5]:
# Save the trained model
model.save('lstm_water_availability_model.h5')

# Save encoders and scalers
import pickle

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

with open('scalers.pkl', 'wb') as f:
    pickle.dump({'scaler_X': scaler_X, 'scaler_y': scaler_y}, f)


In [6]:
# Load the trained model
from tensorflow.keras.models import load_model
model = load_model('lstm_water_availability_model.h5')

# Load encoders and scalers
with open('label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

with open('scalers.pkl', 'rb') as f:
    scalers = pickle.load(f)
scaler_X, scaler_y = scalers['scaler_X'], scalers['scaler_y']

# Perform inference
def perform_inference(year, district, name, month, rainfall):
    try:
        # Encode inputs
        encoded_district = label_encoders['res_district'].transform([district])[0]
        encoded_name = label_encoders['res_name'].transform([name])[0]
        encoded_month = label_encoders['res_month'].transform([month])[0]

        # Prepare input data
        input_data = np.array([[year, rainfall, encoded_district, encoded_month]])
        input_scaled = scaler_X.transform(input_data)
        input_scaled = input_scaled.reshape((1, 1, input_scaled.shape[1]))

        # Make prediction
        predicted_scaled = model.predict(input_scaled)
        predicted_actual = scaler_y.inverse_transform(predicted_scaled)

        return float(predicted_actual[0][0])
    except Exception as e:
        print("Error during inference:", e)
        print(traceback.format_exc())

# Example inference
year = 2025
district = "Nellore"  # Replace with an actual district name
name = "Nellore Anicut"    # Replace with an actual reservoir name
month = "july"               # Replace with an actual month
rainfall = 435.6               # Replace with actual rainfall value

predicted_availability = perform_inference(year, district, name, month, rainfall)
print("Predicted Water Availability:", predicted_availability)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
Predicted Water Availability: 1.387837290763855


In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Predict on the test set
y_pred_scaled = model.predict(X_test)

# Inverse transform the predictions and actual values to original scale
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_actual = scaler_y.inverse_transform(y_test)

# Calculate performance metrics
mse = mean_squared_error(y_actual, y_pred)
mae = mean_absolute_error(y_actual, y_pred)

print("Model Performance on Test Set:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Optional: Compute R-squared score
from sklearn.metrics import r2_score
r2 = r2_score(y_actual, y_pred)
print(f"R-squared Score: {r2:.4f}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  
Model Performance on Test Set:
Mean Squared Error (MSE): 0.0003
Mean Absolute Error (MAE): 0.0075
R-squared Score: 0.9001
